<a href="https://colab.research.google.com/github/TranThiDieuHien/pythonbasics/blob/master/huong_dan_gia_dinh_ANOVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [153]:
from scipy import stats
import numpy as np
import pandas as pd
from scipy.stats import bartlett

PHÂN TÍCH ANOVA MỘT CHIỀU

Phân tích ANOVA chấp nhận các giả định sau:

(1) Tất cả các nhóm đều độc lập với nhau

(2) Dữ liệu của mỗi nhóm đều tuân theo phân phối chuẩn. Tính chất này có thể được kiểm định bằng phương pháp Shapiro-Wilks.

(3) Phương sai (variance) của các nhóm bằng nhau. Tính chất này có thể được kiểm định bằng phương pháp kiểm định Bartlett hay Levene.

Nếu dữ liệu hiện có thỏa mãn các tính chất trên thì chúng ta mới tiến hành kiểm định ANOVA. Do vậy trước khi tiến hành kiểm định ANOVA, chúng ta cần thực hiện kiểm tra tập dữ liệu cần thỏa mãn 3 tính chất trên.
    

In [154]:
#Bước 1: Hãy tạo một tập dữ liệu mẫu sample_1 với kích thước 100 tuân theo phân phối chuẩn, cho mean = 20, std = 0.2
sample_1 = stats.norm.rvs(loc=20, scale=0.2, size=100)
print(sample_1[:10])

[20.11693934 19.89386172 19.96217058 20.0810106  20.09724138 20.04748434
 19.97757972 19.70471335 19.6094493  19.90992411]


In [155]:
#Bước 2: hãy tạo tập dữ liệu mẫu sample_2 với kích thước 100 tuân theo phân phối Poisson
sample_2 = stats.poisson.rvs(loc=20, mu = 1, size = 100)
print(sample_2[:10])

[21 21 20 20 22 21 21 20 21 21]


Hãy tham khảo các hướng dẫn sau để kiểm định tính chất thứ 2 bằng phương pháp Shapiro-Wilks đối với sample_1 và sample_2

(1) [Tài liệu tham khảo scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)

(2) [Hướng dẫn cụ thể của statology](https://www.statology.org/shapiro-wilk-test-python/)

In [156]:
#Code kiểm định tính chất thứ 2 của sample_1 và sample_2
# thử nghiệm Shapiro-Wilk để xác định xem một mẫu có đến từ phân phối chuẩn hay không.
print("Thử nghiệm Shapiro-Wilk tuân theo phân phối chuẩn")
shapiro_test1 = stats.shapiro(sample_1) 
print("Statistic =", shapiro_test1[0], "\n",
      "p value =", shapiro_test1[1])
print("-----------------------------------------")

print("Thử nghiệm Shapiro-Wilk tuân theo phân phối Poisson")
shapiro_test2 = stats.shapiro(sample_2)
print("Statistic =", shapiro_test2[0], "\n",
      "p value =", shapiro_test2[1])

Thử nghiệm Shapiro-Wilk tuân theo phân phối chuẩn
Statistic = 0.9749535918235779 
 p value = 0.05350716412067413
-----------------------------------------
Thử nghiệm Shapiro-Wilk tuân theo phân phối Poisson
Statistic = 0.8053423762321472 
 p value = 3.6244915402328104e-10


Hãy tham khảo các hướng dẫn sau để kiểm định tính chất thứ 3 bằng kiểm định Bartlett hoặc kiểm định Levene

(1) [Hướng dẫn tổng quát](https://www.marsja.se/levenes-bartletts-test-of-equality-homogeneity-of-variance-in-python/)

(2) [Hướng dẫn ở stackoverflow](https://stackoverflow.com/questions/36141254/how-to-test-for-homoscedasticity-having-the-same-population-variance-in-python)

(3) Sử dụng bài giảng về kiểm định Bartlett và Levene ở buổi trước

In [146]:
#Code kiểm định tính đồng nhất về phương sai của các nhóm

Example: An University have 4 Majors: Business, Psych, Math, English. We will consider the level of frustration of 100 students with each major.

- $\mu_{1}$ : level of frustration of Business Majors
- $\mu_{2}$ : level of frustration of English Majors
- $\mu_{3}$ : level of frustration of Math Majors
- $\mu_{4}$ : level of frustration of Psych Majors

The size of each sample is 35. Assume that Y(level of frustration) = 10

In [157]:
#Sinh ngẫu nhiên
Business = np.random.choice(10, 35)
Math = np.random.choice(10, 35)
English = np.random.choice(10, 35)
Psych = np.random.choice(10, 35)
#Tạo dataframe
list_of_tuples = list(zip(Business, English, Math, Psych))
df2 = pd.DataFrame(list_of_tuples, columns = ["Business Majors", "English Majors", "Math Majors", "Psych Majors"])
print(df2.head())
#Lấy từng nhóm
Bus = df2['Business Majors'].unique()
Eng = df2['English Majors'].unique()
Math = df2['Math Majors'].unique()
Psy = df2['Psych Majors'].unique()
alpha = 0.05


   Business Majors  English Majors  Math Majors  Psych Majors
0                0               8            8             0
1                7               1            2             8
2                1               5            4             5
3                6               7            2             1
4                0               4            5             6


In [158]:
#Kiểm định bằng bartlett
stat, pvalue = stats.bartlett(Bus, Eng, Math, Psy)
print("Statistic =", stat, "\n",
      "p value =", pvalue)
if pvalue1 < alpha:
  print("Các quần thể không có phương sai bằng nhau")
else:
  print("Các quần thể có phương sai bằng nhau")
  #Xem phương sai của từng biến
  b, e, m, p = [np.var(x, ddof=1) for x in [Bus, Eng, Math, Psy]]
  print("PS của Bus =", b, "\n",
      "PS của Eng =", e, "\n",
      "PS của Math =", m, "\n",
      "PS của Psy =", p, "\n",)

Statistic = 0.11427463983421088 
 p value = 0.9900710428231514
Các quần thể có phương sai bằng nhau
PS của Bus = 7.5 
 PS của Eng = 9.166666666666666 
 PS của Math = 9.166666666666666 
 PS của Psy = 9.166666666666666 



In [159]:
#Kiểm định bằng Levene
stat1, pvalue1 = stats.levene(Bus, Eng, Math, Psy)
print("Statistic =", stat1, "\n",
      "p value =", pvalue1)
if pvalue1 < alpha:
  print("Các quần thể không có phương sai bằng nhau")
else:
  print("Các quần thể có phương sai bằng nhau")
  #Xem phương sai của từng biến
  b, e, m, p = [np.var(x, ddof=1) for x in [Bus, Eng, Math, Psy]]
  print("PS của Bus =", b, "\n",
      "PS của Eng =", e, "\n",
      "PS của Math =", m, "\n",
      "PS của Psy =", p, "\n",)

Statistic = 0.0824849170437405 
 p value = 0.9691292600784037
Các quần thể có phương sai bằng nhau
PS của Bus = 7.5 
 PS của Eng = 9.166666666666666 
 PS của Math = 9.166666666666666 
 PS của Psy = 9.166666666666666 

